# Isolated XEB

In [ ]:
import cirq
import numpy as np

## Set up Random Circuits

We create a set of 10 random, two-qubit `circuits`.

In [ ]:
from cirq.experiments import random_quantum_circuit_generation as rqcg
circuits = rqcg.generate_library_of_2q_circuits(
    n_library_circuits=10,
    two_qubit_gate=cirq.ISWAP**0.5,
    q0=cirq.GridQubit(4,4),
    q1=cirq.GridQubit(4,5),
)
print(len(circuits))

In [ ]:
# We will truncate to these lengths
cycle_depths = np.arange(3, 100, 20)
cycle_depths

In [ ]:
sampler = cirq.DensityMatrixSimulator(noise=cirq.depolarize(5e-3))

In [ ]:
# import cirq.google as cg
# import os
# os.environ['GOOGLE_CLOUD_PROJECT'] = 'google.com:quantum-experiments'
# sampler = cg.get_engine_sampler('rainbow', gate_set_name='sqrt_iswap')

# device = cg.get_engine_device('rainbow')
# device

# import cirq.contrib.routing as ccr
# graph = ccr.xmon_device_to_graph(device)
# nx.draw_networkx(graph)

In [ ]:
from cirq.experiments.fidelity_estimation import sample_2q_xeb_circuits
sampled_df = sample_2q_xeb_circuits(
    sampler=sampler,
    circuits=circuits,
    cycle_depths=cycle_depths,
)
sampled_df

In [ ]:
from cirq.experiments.fidelity_estimation import benchmark_2q_xeb_fidelities
fids = benchmark_2q_xeb_fidelities(
    sampled_df=sampled_df,
    circuits=circuits,
    cycle_depths=cycle_depths,
)
fids

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt

xx = np.linspace(0, fids['cycle_depth'].max())
plt.plot(xx, (1-5e-3)**(4*xx), label=r'Exponential Reference')

def _p(fids):
    plt.plot(fids['cycle_depth'], fids['fidelity'], 'o-', label=fids.name)

fids['group'] = 'Sampled'
fids.groupby('group').apply(_p)

plt.ylabel('Circuit fidelity')
plt.xlabel('Cycle Depth $d$')
plt.legend(loc='best')

## Optimize `PhasedFSimGate` parameters

We know what circuits we requested, and in this simulated example, we know what coherent error has happened. But in a real experiment, there is likely unknown coherent error that you would like to characterize. Therefore, we make the five angles in `PhasedFSimGate` free parameters and use a classical optimizer to find which set of parameters best describes the data we collected from the noisy simulator (or device, if this was a real experiment).

fids_opt = simulate_2q_xeb_fids(sampled_df, pcircuits, cycle_depths, param_resolver={'theta': -np.pi/4, 'phi': 0.1})

In [ ]:
import multiprocessing
pool = multiprocessing.get_context('spawn').Pool()

In [ ]:
from cirq.experiments.fidelity_estimation import \
    parameterize_phased_fsim_circuit, optimize_xeb, SqrtISwapXEBOptions

options = SqrtISwapXEBOptions()
pcircuits = [parameterize_phased_fsim_circuit(circuit, options) for circuit in circuits]
res = optimize_xeb(sampled_df, pcircuits, cycle_depths, options, pool=pool)

In [ ]:
res

In [ ]:
_, names = options.get_initial_simplex_and_names()
final_params = dict(zip(names, res.x))
final_params

In [ ]:
fids_opt = simulate_2q_xeb_fidelities(
    sampled_df, pcircuits, cycle_depths, 
    param_resolver=final_params)

In [ ]:
xx = np.linspace(0, fids['cycle_depth'].max())
p_depol = 5e-3 # from above
plt.plot(xx, (1-p_depol)**(4*xx), label=r'Exponential Reference')
plt.axhline(1, color='grey', ls='--')

plt.plot(fids['cycle_depth'], fids['fidelity'], 'o-', label='Raw')
plt.plot(fids_opt['cycle_depth'], fids_opt['fidelity'], 'o-', label='Refit')

plt.ylabel('Circuit fidelity')
plt.xlabel('Cycle Depth')
plt.legend(loc='best')
plt.tight_layout()